In [2]:
#import pathlib
#import sys
#import itertools
#import numpy as np

#low, high = 2, 900

#gen = (x for x in range(1000))
#%timeit list(itertools.islice(gen, low, high))

#print(sys.getsizeof(list(itertools.islice(gen, low, high))))

#gen = [x for x in range(1000)]
#%timeit gen[low:high]

#print(sys.getsizeof(gen[low:high]))

#gen = np.arange(1000)
#%timeit gen[low:high]

#print(sys.getsizeof(gen[low:high]))

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from IPython.utils import io
import os
import json

import warnings
warnings.filterwarnings('ignore')


import mfNRcatpy.base as bs
import mfNRcatpy.utils as ut
import mfNRcatpy.utils_wf as utw
import mfNRcatpy.snr as snr
#import mfNRcatpy.viz as viz
import mfNRcatpy.catalog_tools as cat
#from mfNRcatpy.lal_wf import *

import mfNRcatpy.core_wf as core

In [4]:
#from eosdb import catalog

In [5]:
deltat = 1.0/(40000.0)
inclination = 0.
phiref = 0.
dist_mpc = 100.0

src_core = {'inclination':inclination,'phiref':phiref, 'dist_mpc':dist_mpc} 


db_path_CoRe = os.path.expanduser('~/REPOS3/mfNRcatpy/CoRe_DB')
core_list = cat.find_infd(db_path_CoRe, ext = '.h5', mode='core')
core_list_eos = core.core_eos_group(db_path_CoRe, eos='H4',key='id_eos')[:]
#print(core_list)
print(core_list_eos)

['BAM:0035', 'BAM:0036', 'BAM:0037', 'BAM:0038', 'BAM:0039', 'BAM:0040', 'BAM:0041', 'BAM:0042', 'BAM:0043', 'BAM:0044', 'BAM:0045', 'BAM:0046', 'BAM:0047', 'BAM:0048', 'BAM:0049', 'BAM:0050', 'BAM:0051', 'BAM:0052', 'BAM:0053', 'BAM:0054', 'BAM:0055', 'BAM:0056', 'BAM:0057']


In [6]:
path = '../files/'

with open(path+'wf_bank_core.txt','w') as outfile:
    data_corewf = {}
    data_corewf['db_path']= db_path_CoRe
    data_corewf['ex_param'] = src_core
    data_corewf['sims'] = {}
    for key in core_list:
        data_corewf['sims'][key]={}
    #print(data_corewf)
    json.dump(data_corewf, outfile)

In [7]:
with open('../files/wf_bank_core.txt') as json_file:
    waveform_bank_core = json.load(json_file)
    
result = waveform_bank_core.copy()
print(result['sims']['BAM:0001'])

{}


In [8]:
def whatever(name , db_path, src, A):
    #print(name)
    wf = core.CoRe_NR_wf22(db_path, name, src,resol='best')
    wf_p0 = ut.find_postm(wf)
    
    result = A.copy()
    result['sims'][name]={}
    
    result['sims'][name]['wf_d'] = wf_p0.dur
    result['sims'][name]['wf_d_wei'] = wf_p0.amp_wei_time()
    result['sims'][name]['wf_f_wei'] = wf_p0.amp_wei_freq()
    result['sims'][name]['mdta'] = dict(wf_p0.mdta)
    
    EOS_name = result['sims'][name]['mdta']['eos']
    if EOS_name in utw.EOS_masses().keys():
        result['sims'][name]['mdta']['maxm_TOV'] = utw.EOS_masses()[EOS_name]['maxm_TOV']
    else:
        result['sims'][name]['mdta']['maxm_TOV'] = np.nan
    return result

f = lambda name: whatever(name, db_path_CoRe, src_core, result)

In [10]:
%%time
r = list(map(f, core_list[:]))

print(r)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
params = {'text.usetex' : True,
          'font.size' : 11,
          'font.family' : 'lmodern'
           }
plt.rcParams.update(params) 

db_path_CoRe = os.path.expanduser('~/CoRe_DB')

In [ ]:
deltat = 1.0/(160000.0)
inclination = 0
long = 0
dist_mpc = 100.0

src_lal = {'inclination':inclination,'phiref':long, 'dist_mpc':dist_mpc}
src_core = {'inclination':inclination,'phiref':long, 'dist_mpc':dist_mpc}

In [ ]:
lal_label, core_label = ['/home/gthth/REPOS1/lvcnr-matter/BNS/parma/EOS_SLy/mgrav_1.44_1.39/SLy_Mtot_2829_q_104_dx_369_D44_Feo_2016cbs/SLy_Mtot_2829_q_104_dx_369_D44_Feo_2016cbs.h5' ,
                         'BAM:0024'        
                        ]

In [ ]:
wf0 = lal_NR_wf1(lal_label,2,2,deltat, src_lal)
wf_core0 = CoRe_NR_wf22(db_path_CoRe,core_label, src_core, resol='best')

# 1. Importing waveform banks and template banks

In [ ]:
with open('../files/wf_bank_core.txt') as json_file:
    waveform_bank_core = json.load(json_file)
    
with open('../files/wf_bank_lal.txt') as json_file:
    waveform_bank_lal = json.load(json_file)

In [ ]:
path = '../results/'

name1 = f'CoRe50x50_(1000, 3500).txt'
name2 = f'LAL50x50_(1000, 3500).txt'


with open(path+name1) as json_file:
    print(f'the file "{name1}" has been loaded')
    CORE = json.load(json_file)

with open(path+name2) as json_file:
    print(f'the file "{name2}" has been loaded')
    LAL = json.load(json_file)

In [ ]:
LAL1, mapping = cat.relabler_cat(LAL, 'lal:', maxdigs=None)
label = 'lal:0045'
name = mapping[label]
print(f'the simulation: \n{name} \n\nwas assigned the label {label}')

In [ ]:
JOINT = CORE.copy()
JOINT.update(LAL1)

sim1,sim2 = 'lal:0001', 'BAM:0092'
print(sim1+':\n \n',JOINT[sim1],'\n \n')
print(sim2+':\n \n',JOINT[sim2],'\n \n')

# 1. Reading the catalog using signal features

In [ ]:
f = lambda x: x #---> this is useful in cases one wants to use sthing like 1-Sopt/Srec 
g = lambda y: y*1e3

ax, R0 = cat.scatter_cat(JOINT, f, g, 
                ['delta'],       #---> keys of the dict that are needed to compute f, which is used as x axis
                ['wf_d'],       #---> keys of the dict that are needed to compute g, which is used as y axis
                
                bounds=[[1.5, 2, 0, 5]
                        ], 
                boundsv=None, 
                mdta = None
                   )

ax.set_xlabel(r'$\delta=\mathrm{\frac{M_{g_{tot}}}{M_{TOV}}}$', fontsize=20)
ax.set_ylabel(r'$\mathrm{d_{postm} [ms]}$', fontsize=20)
ax.grid(linestyle='dashed', linewidth=0.2)

plt.tight_layout()
plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/res0.pdf')


f = lambda x: x #---> this is useful in cases one wants to use sthing like 1-Sopt/Srec 
g = lambda y: 2*y*1e3

ax, R1 = cat.scatter_cat(JOINT, f, g, 
                ['delta'],       #---> keys of the dict that are needed to compute f, which is used as x axis
                ['wf_d_wei'],       #---> keys of the dict that are needed to compute g, which is used as y axis
                
                bounds=[[1.5, 2, 0, 5]
                        ], 
                boundsv=None, 
                mdta = None
                   )

ax.set_xlabel(r'$\delta=\mathrm{\frac{M_{g_{tot}}}{M_{TOV}}}$', fontsize=20)
ax.set_ylabel(r'$\mathrm{2\cdot d_{wei} [ms]}$', fontsize=20)
ax.grid(linestyle='dashed', linewidth=0.2)

print(R0, '\n\n', R1)

In [ ]:
f = lambda x: x #---> this is useful in cases one wants to use sthing like 1-Sopt/Srec 
g = lambda y: y*1e3

ax, R0 = cat.scatter_cat(JOINT, f, g, 
                ['M_r'],       #---> keys of the dict that are needed to compute f, which is used as x axis
                ['wf_d'],       #---> keys of the dict that are needed to compute g, which is used as y axis
                
                bounds=None, 
                boundsv=None, 
                mdta = None
                   )

ax.set_xlabel(r'$\mathrm{q=\frac{m_1}{m_2}}$', fontsize=20)
ax.set_ylabel(r'$\mathrm{d_{postm} [ms]}$', fontsize=20)
ax.grid(linestyle='dashed', linewidth=0.2)




f = lambda x: x #---> this is useful in cases one wants to use sthing like 1-Sopt/Srec 
g = lambda y: 2*y*1e3

ax, R1 = cat.scatter_cat(JOINT, f, g, 
                ['M_r'],       #---> keys of the dict that are needed to compute f, which is used as x axis
                ['wf_d_wei'],       #---> keys of the dict that are needed to compute g, which is used as y axis
                
                bounds=None,
                boundsv=['M_r', 1.6, 3], 
                mdta = None
                   )

ax.set_xlabel(r'$\mathrm{q=\frac{m_1}{m_2}}$', fontsize=20)
ax.set_ylabel(r'$\mathrm{2\cdot d_{wei} [ms]}$', fontsize=20)
ax.grid(linestyle='dashed', linewidth=0.2)

plt.tight_layout()
plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/Mr.pdf')


#print(R0, '\n\n', R1)
print(R1)

In [ ]:
#### 
from matplotlib.ticker import FormatStrFormatter

f = lambda x: x #---> this is useful in cases one wants to use sthing like 1-Sopt/Srec 
g = lambda y: -1*y

ax, R = cat.scatter_cat(JOINT, f, g, 
                ['E'],       #---> keys of the dict that are needed to compute f, which is used as x axis
                ['Jz'],       #---> keys of the dict that are needed to compute g, which is used as y axis
                
                bounds=[[8e-6, 1e-5, 2.5e-4, 3e-4]], 
                boundsv=None, 
                mdta = True
                   )

print(R)
ax.yaxis.set_major_formatter(FormatStrFormatter('%1.2e'))
ax.xaxis.set_major_formatter(FormatStrFormatter('%1.2e'))

ax.set_xlabel(r'$\mathrm{E[M_\odot]}$', fontsize=20)
ax.set_ylabel(r'$\mathrm{J_z[M^2_\odot]}$', fontsize=20)
ax.grid(linestyle='dashed', linewidth=0.2)

In [ ]:
f = lambda x: x*1e3 #---> this is useful in cases one wants to use sthing like 1-Sopt/Srec 
g = lambda y: 2*y*1e3

ax, R = cat.scatter_cat(JOINT, f, g, 
                ['wf_d'],       #---> keys of the dict that are needed to compute f, which is used as x axis
                ['wf_d_wei'],       #---> keys of the dict that are needed to compute g, which is used as y axis
                
                bounds=[[48, 80, 0, 65]
                    
                ], 
                boundsv=None, 
                mdta = None
                   )

ax.set_xlabel(r'$\mathrm{d_{postm} [ms]}$', fontsize=20)
ax.set_ylabel(r'$\mathrm{2\cdot d_{wei} [ms]}$', fontsize=20)
ax.grid(linestyle='dashed', linewidth=0.2)

plt.tight_layout()
plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/res1.pdf')

#print(JOINT['BAM:0074'])
print('Highly eccentric waveforms deviate the most from the linear trend:\n\n',R['r'])

In [ ]:
f = lambda x: x #---> this is useful in cases one wants to use sthing like 1-Sopt/Srec 
g = lambda y: y*1e3

ax, R0 = cat.scatter_cat(JOINT, f, g, 
                ['wf_f_wei'],       #---> keys of the dict that are needed to compute f, which is used as x axis
                ['wf_d'],       #---> keys of the dict that are needed to compute g, which is used as y axis
                
                bounds=[[3000, 4000, 0, 10],
                        [0, 2000, 0, 10],
                        
                        ], 
                boundsv=None, 
                mdta = None
                   )

ax.set_ylabel(r'$\mathrm{d_{postm} [ms]}$', fontsize=20)
ax.set_xlabel(r'$\mathrm{f_{wei} [Hz]}$', fontsize=20)
ax.grid(linestyle='dashed', linewidth=0.2)

f = lambda x: x #---> this is useful in cases one wants to use sthing like 1-Sopt/Srec 
g = lambda y: 2*y*1e3

ax, R1 = cat.scatter_cat(JOINT, f, g, 
                ['wf_f_wei'],       #---> keys of the dict that are needed to compute f, which is used as x axis
                ['wf_d_wei'],       #---> keys of the dict that are needed to compute g, which is used as y axis
                
                bounds=[[3000, 4000, 0, 10],
                        [0, 2000, 0, 10],
                    
                ], 
                boundsv=None, 
                mdta = None
                   )

print(R0)
print(R1)
ax.set_ylabel(r'$\mathrm{2\cdot d_{wei} [ms]}$', fontsize=20)
ax.set_xlabel(r'$\mathrm{f_{wei} [Hz]}$', fontsize=20)
ax.grid(linestyle='dashed', linewidth=0.2)

plt.tight_layout()
plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/res2.pdf')

# 2. Reading the catalog using matched filtering quantities

In [ ]:
col = []#'BAM:0001', 'BAM:0011', 'BAM:0016', 'BAM:0031', 'BAM:0032',
      #'BAM:0047', 'BAM:0052', 'BAM:0058', 'BAM:0069', 'BAM:0095',
      #'BAM:0099', 'BAM:0100', 'BAM:0101', 'BAM:0120', 'BAM:0125',
      #'THC:0008', 'THC:0009', 'THC:0024', 'THC:0025', 'THC:0026',
      #'THC:0027', 'THC:0028', 'THC:0033', 'THC:0034', 'THC:0035',
      #'/home/gthth/REPOS1/lvcnr-matter/BNS/parma/EOS_SLy/mgrav_1.44_1.39/SLy_Mtot_2829_q_104_dx_369_D44_Feo_2016cbs/SLy_Mtot_2829_q_104_dx_369_D44_Feo_2016cbs.h5', 
      #'/home/gthth/REPOS1/lvcnr-matter/BNS/parma/EOS_SLy/mgrav_1.44_1.44/SLy_Mtot_2886_q_100_dx_369_D40_DePietri_2015lya/SLy_Mtot_2886_q_100_dx_369_D40_DePietri_2015lya.h5', 
      #'/home/gthth/REPOS1/lvcnr-matter/BNS/parma/EOS_SLy/mgrav_1.44_1.44/SLy_Mtot_2886_q_100_dx_369_D40_Maione_2017aux/SLy_Mtot_2886_q_100_dx_369_D40_Maione_2017aux.h5']
ecc =[]#['BAM:0074','BAM:0075','BAM:0076','BAM:0077','BAM:0078','BAM:0079',
    #'BAM:0112','BAM:0113','BAM:0114','BAM:0115','BAM:0116','BAM:0117',
    # 'BAM:0118', 'BAM:0119']
L = []#'BAM:0017', 'BAM:0018', 'BAM:0019', 'BAM:0020', 'BAM:0021',
     #'BAM:0053', 'BAM:0054', 'BAM:0055', 'BAM:0056', 'BAM:0057',
     #'BAM:0081', 'BAM:0082', 'BAM:0083', 'BAM:0084', 'BAM:0085',
     #'BAM:0086', 'BAM:0087', 'BAM:0091', 'BAM:0093', 'BAM:0094',
     #'BAM:0124', 'BAM:0127', 'BAM:0128']

In [ ]:
JOINT1 = JOINT.copy()
excep = col + ecc + L
for name in excep:
    JOINT1.pop(name)

In [ ]:
low =  [0.4, 0.65]
high = [0.8, 1.05]
            


f = lambda srec, sopt: srec/sopt #---> this is useful in cases one wants to use sthing like 1-Sopt/Srec 
g = lambda sopt: sopt


fig = plt.figure(dpi=1200)
ax = fig.add_subplot(1,1,1)
line1 = ax.axvspan(low[0], low[-1], color='red', alpha=0.5, label='"poor" recovery\n using monochromatic')
line2 = ax.axvspan(high[0], high[-1], color='green', alpha=0.5, label='"good" recovery\n using monochromatic')
ax.axhline(8, color='black', linestyle='dashed')
plt.text(low[0]+0.15, 11, 'Detectable waveforms @100MPc using ET.', fontsize=10)
plt.text(low[0]+0.15, 5, 'Non-detectable waveforms @100MPc using ET.', fontsize=10)

ax.set_ylabel(r'$S_{opt}$')
ax.set_xlabel(r'$parameter$')
ax.set_xlim(low[0], high[-1])
ax.set_ylim(3,14)
ax.set_xticklabels([])


fig.legend([line1, line2], 
           ['"poor" recovery\n using monochromatic', '"good" recovery\n using monochromatic'],
          loc='upper center',
          fontsize=8)

plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/schematics.pdf')

In [ ]:
fig1 = plt.figure(dpi=1200)
ax0, ax1 = [fig1.add_subplot(1,2,1),
            fig1.add_subplot(1,2,2)]

ax0.axhline(8, color='black', linestyle='dashed')
ax0.set_ylabel(r'$S_{opt}$')
ax0.set_xlim(low[0], high[-1])
ax0.set_ylim(3,14)
ax0.set_xticklabels([])

ax1.axhline(8, color='black', linestyle='dashed')
ax1.set_xlabel(r'$\alpha=\frac{\rho_{rec}}{\rho_{opt}}$')
ax1.set_xlim(low[0], high[-1])
ax1.set_ylim(8,14)

ax0.set_title('Whole dataset')
ax1.set_title('Above treshold')


_, R0 = cat.scatter_cat(JOINT1, f, g, 
                ['SNR_rec','SNR_opt'],       #---> keys of the dict that are needed to compute f, which is used as x axis
                ['SNR_opt'],       #---> keys of the dict that are needed to compute g, which is used as y axis
                
                bounds=[[low[0], low[-1], 0, 20],  # bounds requires a list of lists as input
                        [high[0], high[-1], 0, 20],
                        [low[-1], high[0], 0,20]
                        ], 
                boundsv=None,
                ax = ax0
                        
                   )

_, R1 = cat.scatter_cat(JOINT1, f, g, 
                ['SNR_rec','SNR_opt'],       #---> keys of the dict that are needed to compute f, which is used as x axis
                ['SNR_opt'],       #---> keys of the dict that are needed to compute g, which is used as y axis
                
                bounds=[[low[0], low[-1], 8, 20],  # bounds requires a list of lists as input
                        [high[0], high[-1], 8, 20],
                        [low[-1], high[0], 8, 20]
                        ], 
                boundsv=None,
                ax = ax1
                        
                   )

plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/alpha_scatter.pdf')

In [ ]:
# d_postm -> not needed

#fig1 = plt.figure(dpi=1200)
#ax0, ax1 = [fig1.add_subplot(1,2,1),
#            fig1.add_subplot(1,2,2)]

#ax0.yaxis.set_major_locator(MaxNLocator(integer=True))
#ax1.yaxis.set_major_locator(MaxNLocator(integer=True))

#ax0.set_ylabel(r'$Count$')
#ax1.set_xlabel(r'$d_{postm}$[ms]')


#f = lambda x:x*1e3
#b = np.linspace(0,80, 25, endpoint=True)
#_ = cat.hist_cat(JOINT1, R0, f, var=['wf_d'],ax=ax0, varmdta=None, bins=b);
#_ = cat.hist_cat(JOINT1, R1, f, var=['wf_d'],ax=ax1, varmdta=None, bins=b);



fig1 = plt.figure(dpi=1200)
ax0, ax1 = [fig1.add_subplot(1,2,1),
            fig1.add_subplot(1,2,2)]

ax0.set_title('Whole dataset')
ax1.set_title('Above treshold')

ax0.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_major_locator(MaxNLocator(integer=True))

ax0.set_ylabel(r'$Count$')
ax1.set_xlabel(r'$2\cdot d_{wei}$[ms]')


f = lambda x:2*x*1e3
b = np.linspace(0,80, 25, endpoint=True)

_ = cat.hist_cat(JOINT1, R0, f, var=['wf_d_wei'],ax=ax0, varmdta=None, bins=b);
_ = cat.hist_cat(JOINT1, R1, f, var=['wf_d_wei'],ax=ax1, varmdta=None, bins=b);


plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/alpha_dhist.pdf')

fig1 = plt.figure(dpi=1200)
ax0, ax1 = [fig1.add_subplot(1,2,1),
            fig1.add_subplot(1,2,2)]

ax0.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_major_locator(MaxNLocator(integer=True))

ax0.set_ylabel(r'$Count$')
ax1.set_xlabel(r'$f_{wei}$[Hz]')


f = lambda x:x
b= np.linspace(1000, 3500, 25)
_ = cat.hist_cat(JOINT1, R0, f, var=['wf_f_wei'],ax=ax0, varmdta=None, bins=b);
_ = cat.hist_cat(JOINT1, R1, f, var=['wf_f_wei'],ax=ax1, varmdta=None, bins=b);

plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/alpha_fhist.pdf')


In [ ]:
fig1 = plt.figure(dpi=1200)
ax0, ax1 = [fig1.add_subplot(1,2,1),
            fig1.add_subplot(1,2,2)]

ax0.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_major_locator(MaxNLocator(integer=True))

ax0.set_ylabel(r'$Count$')
ax1.set_xlabel(r'$\beta$')


f = lambda x, y:x/y
b = np.linspace(0,1, 25, endpoint=True)
_ = cat.hist_cat(JOINT1, R0, f, var=['best_d','wf_d'],ax=ax0, varmdta=None, bins=b);
_ = cat.hist_cat(JOINT1, R1, f, var=['best_d','wf_d'],ax=ax1, varmdta=None, bins=b);

plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/alpha_betahist.pdf')

# beta_wei -> not needed

#fig1 = plt.figure(dpi=1200)
#ax0, ax1 = [fig1.add_subplot(1,2,1),
#            fig1.add_subplot(1,2,2)]

#ax0.yaxis.set_major_locator(MaxNLocator(integer=True))
#ax1.yaxis.set_major_locator(MaxNLocator(integer=True))

#ax0.set_ylabel(r'$Count$')
#ax1.set_xlabel(r'$\beta_{wei}$')


#f = lambda x, y:x/(2*y)
#b = np.linspace(0,1, 25, endpoint=True)
#_ = cat.hist_cat(JOINT1, R0, f, var=['best_d','wf_d_wei'],ax=ax0, varmdta=None, bins=b);
#_ = cat.hist_cat(JOINT1, R1, f, var=['best_d','wf_d_wei'],ax=ax1, varmdta=None, bins=b);



fig1 = plt.figure(dpi=1200)
ax0, ax1 = [fig1.add_subplot(1,2,1),
            fig1.add_subplot(1,2,2)]

ax0.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_major_locator(MaxNLocator(integer=True))

ax0.set_ylabel(r'$Count$')
ax1.set_xlabel(r'$\gamma_{wei}$')


f = lambda x, y:x/y
b = np.linspace(0,1.5, 25, endpoint=True)
_ = cat.hist_cat(JOINT1, R0, f, var=['best_f','wf_f_wei'],ax=ax0, varmdta=None, bins=b);
_ = cat.hist_cat(JOINT1, R1, f, var=['best_f','wf_f_wei'],ax=ax1, varmdta=None, bins=b);

plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/alpha_gammahist.pdf')


In [ ]:
fig1 = plt.figure(dpi=1200)
ax0, ax1 = [fig1.add_subplot(1,2,1),
            fig1.add_subplot(1,2,2)]

ax0.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_major_locator(MaxNLocator(integer=True))

ax0.set_ylabel(r'$Count$')
ax1.set_xlabel(r'$\delta$')

f = lambda x, y: x/y
b = np.linspace(0.7,2, 25, endpoint=True)
_ = cat.hist_cat(JOINT1, R0, f, var=None,ax=ax0, varmdta=['M_t','maxm_TOV'], bins=b);
_ = cat.hist_cat(JOINT1, R1, f, var=None,ax=ax1, varmdta=['M_t','maxm_TOV'], bins=b);

plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/alpha_deltahist.pdf')


fig1 = plt.figure(dpi=1200)
ax0, ax1 = [fig1.add_subplot(1,2,1),
            fig1.add_subplot(1,2,2)]

ax0.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_major_locator(MaxNLocator(integer=True))

ax0.set_ylabel(r'$Count$')
ax1.set_xlabel(r'$q$')

f = lambda x: x
b = np.linspace(0.7,2.5, 25, endpoint=True)
_ = cat.hist_cat(JOINT1, R0,f,ax=ax0, var=None, varmdta=['M_r'], bins=b);
_ = cat.hist_cat(JOINT1, R1, f,ax=ax1, var=None, varmdta=['M_r'], bins=b);

plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/alpha_qhist.pdf')


fig1 = plt.figure(dpi=1200)
ax0, ax1 = [fig1.add_subplot(1,2,1),
            fig1.add_subplot(1,2,2)]

ax0.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_major_locator(MaxNLocator(integer=True))

ax0.set_ylabel(r'$Count$')
ax1.set_xlabel(r'$\chi_{eff}$')

f = lambda x: x
b = np.linspace(0.,0.5, 25, endpoint=True)
_ = cat.hist_cat(JOINT1, R0,f,ax=ax0, var=None, varmdta=['Xeff'], bins=b);
_ = cat.hist_cat(JOINT1, R1, f,ax=ax1, var=None, varmdta=['Xeff'], bins=b);

plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/alpha_chihist.pdf')


fig1 = plt.figure(dpi=1200)
ax0, ax1 = [fig1.add_subplot(1,2,1),
            fig1.add_subplot(1,2,2)]

ax0.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_major_locator(MaxNLocator(integer=True))


ax0.set_ylabel(r'$Count$')
ax1.set_xlabel(r'$\tilde{\Lambda}$')

f = lambda x: x
b = np.linspace(0,6700, 25, endpoint=True)
_ = cat.hist_cat(JOINT1, R0,f,ax=ax0, var=None, varmdta=['lambda_tilde'], bins=b);
_ = cat.hist_cat(JOINT1, R1, f,ax=ax1, var=None, varmdta=['lambda_tilde'], bins=b);

plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/alpha_lambdahist.pdf')


fig1 = plt.figure(dpi=1200)
ax0, ax1 = [fig1.add_subplot(1,2,1),
            fig1.add_subplot(1,2,2)]

ax0.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_major_locator(MaxNLocator(integer=True))

ax0.xaxis.set_major_formatter(FormatStrFormatter('%1.2e'))
ax1.xaxis.set_major_formatter(FormatStrFormatter('%1.2e'))


ax0.set_ylabel(r'$Count$')
ax1.set_xlabel(r'$E[M_\odot]$')

f = lambda x: x
b = np.linspace(0.,1.2e-5, 25, endpoint=True)
_ = cat.hist_cat(JOINT1, R0,f,ax=ax0, var=None, varmdta=['E'], bins=b);
_ = cat.hist_cat(JOINT1, R1, f,ax=ax1, var=None, varmdta=['E'], bins=b);


fig1 = plt.figure(dpi=1200)
ax0, ax1 = [fig1.add_subplot(1,2,1),
            fig1.add_subplot(1,2,2)]

ax0.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_major_locator(MaxNLocator(integer=True))

ax0.xaxis.set_major_formatter(FormatStrFormatter('%1.2e'))
ax1.xaxis.set_major_formatter(FormatStrFormatter('%1.2e'))


ax0.set_ylabel(r'$Count$')
ax1.set_xlabel(r'$J_z[M^2_\odot]$')

f = lambda x: -x
b = np.linspace(4e-5,3e-4, 25, endpoint=True)
_ = cat.hist_cat(JOINT1, R0,f,ax=ax0, var=None, varmdta=['Jz'], bins=b);
_ = cat.hist_cat(JOINT1, R1, f,ax=ax1, var=None, varmdta=['Jz'], bins=b);

In [ ]:
ecc =['BAM:0074','BAM:0075','BAM:0076','BAM:0077','BAM:0078','BAM:0079',
    'BAM:0112','BAM:0113','BAM:0114','BAM:0115','BAM:0116','BAM:0117',
     'BAM:0118', 'BAM:0119']



fig4 = plt.figure(dpi=1200)
ax4 = fig4.add_subplot()

O = []
for name in JOINT:
    sopt = JOINT[name]['SNR_opt'][0]
    srec = JOINT[name]['SNR_rec'][0]
    D = JOINT[name]['wf_d'][0]
    q = JOINT[name]['mdta']['M_r'][0]
    
    
    if D<=5e-3:
        line1=ax4.scatter(srec, sopt, c='green', marker='x',
                    s=10)
    elif q>=1.6: 
        O.append(name)
        line2=ax4.scatter(srec, sopt, c='r', marker='v',s=10)
    elif name in ecc:
        line3=ax4.scatter(srec, sopt, c='black', s=10)
    else:
        ax4.scatter(srec, sopt, c='blue', s=3)
        
plt.legend([line1, line2, line3], 
           [r'$\mathrm{d_{postm}\leq 5ms}$',
            r'$\mathrm{q \geq 1.5}$',
            r'highly eccentric'], loc='upper right',
          fontsize='x-small') 

plt.xlabel(r'$\mathrm{S_{rec}}$', fontsize=20) 
plt.ylabel(r'$\mathrm{S_{opt}}$', fontsize=20)

plt.tight_layout()
#plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/stats/outL3.png')


In [ ]:
fig4 = plt.figure(dpi=1200)
ax4 = fig4.add_subplot()

vline = 0.80

O = []
count80 = 0
countoutL = 0
for name in JOINT:
    sopt = JOINT[name]['SNR_opt'][0]
    srec = JOINT[name]['SNR_rec'][0]
    
    D = JOINT[name]['wf_d'][0]
    q = JOINT[name]['mdta']['M_r'][0]
    chieff = JOINT[name]['mdta']['Xeff'][0]
        
    if srec/sopt>=vline:
        count80+=1
    
    if D<=5e-3:
        line1 = ax4.scatter(srec/sopt, sopt, c='green', 
                    marker='x', s=10)
        if srec/sopt>=vline:
            countoutL+=1
    
    elif q>=1.5: 
        O.append(name)
        line2 = ax4.scatter(srec/sopt, sopt, c='r', s=10, marker='v')
        if srec/sopt>=vline:
            countoutL+=1   
    elif name in ecc:
        line3 = ax4.scatter(srec/sopt, sopt, c='black', s=10)
        if srec/sopt>=vline:
            countoutL+=1  
    else:
        line4 = ax4.scatter(srec/sopt, sopt, c='blue', s=3)
        
plt.legend([line1, line2, line3], 
           [r'$\mathrm{d_{postm}\leq 5ms}$',
            r'$\mathrm{q \geq 1.6}$',
            r'highly eccentric'], loc='upper right',
          fontsize='x-small') 

plt.xlabel(r'$\mathrm{\alpha=\frac{\rho_{rec}}{\rho_{opt}}}$'
           , fontsize=20) 
plt.ylabel(r'$\mathrm{\rho_{opt}}$'
           , fontsize=20)

#plt.title(f'{count80} waveforms, out of which {countoutL/count80 * 100:.0f} percent are either...')
plt.axvline(vline, linestyle='dashed', color='red');
#plt.xlim(0.5,1);
plt.ylim(0,20);

plt.axhline(8, linestyle='dashed', color='red')

plt.tight_layout()

plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/alpha_sum0.pdf')


In [ ]:
fig4 = plt.figure(dpi=1200)
ax4 = fig4.add_subplot()#2,1,1)
#ax5 = fig4.add_subplot(2,1,2)

vline = 0.1
count20 = 0
countoutL = 0
for name in JOINT:
    sopt = JOINT[name]['SNR_opt'][0]
    srec = JOINT[name]['SNR_rec'][0]
    
    q = JOINT[name]['mdta']['M_r'][0]

    
    D = JOINT[name]['wf_d'][0]
    d_b = JOINT[name]['best_d'][0]
    
    
    Fw = JOINT[name]['wf_f_wei'][0]
    f_b = JOINT[name]['best_f'][0]
    
    gamma = f_b/Fw      
    beta = d_b/D
    


    if beta <= 0.1:
        count20+=1
    if q>=1.5: 
        line1 = ax4.scatter(beta, srec/sopt, c='r', marker='v', s=10)
        if beta <= 0.1:
            countoutL+=1

    elif name in ecc:
        line2 = ax4.scatter(beta, srec/sopt, c='black', s=10)
        if beta <= 0.1:
            countoutL+=1
    elif D>30e-3:
        line3 = ax4.scatter(beta, srec/sopt, c='darkorange', s=10)
        if beta <= 0.1:
            countoutL+=1
    else:
        ax4.scatter(beta, srec/sopt, c='blue', s=3)

plt.legend([line1, line2, line3], 
           [r'$\mathrm{q \geq 1.5}$',
            r'highly eccentric', 
            r'$\mathrm{dur_{postm} \geq 25ms}$'], loc='upper right',
          fontsize='x-small') 

plt.xlabel(r'$\beta=\mathrm{\frac{d_{best}}{d_{postm}}}$', fontsize=20) 
plt.ylabel(r'$\mathrm{\alpha=\frac{\rho_{rec}}{\rho_{opt}}}$'
           , fontsize=20)

#plt.title(f'{count20} waveforms, out of which {countoutL/count20 * 100:.0f} percent are either...')
plt.axvline(vline, linestyle='dashed', color='red');
plt.xlim(0, 1.3)

plt.tight_layout()

plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/alpha_sum1.pdf')



In [ ]:
fig4 = plt.figure(dpi=1200)
ax4 = fig4.add_subplot()#2,1,1)
#ax5 = fig4.add_subplot(2,1,2)

vline = 0.1
count20 = 0
countoutL = 0
for name in JOINT:
    sopt = JOINT[name]['SNR_opt'][0]
    srec = JOINT[name]['SNR_rec'][0]
    
    q = JOINT[name]['mdta']['M_r'][0]

    
    D = JOINT[name]['wf_d'][0]
    d_b = JOINT[name]['best_d'][0]
    
    
    Fw = JOINT[name]['wf_f_wei'][0]
    f_b = JOINT[name]['best_f'][0]
    
    gamma = f_b/Fw      
    beta = d_b/D
    


    if beta <= 0.1:
        count20+=1
    if q>=1.5: 
        line1 = ax4.scatter(beta, sopt, c='r', marker='v', s=10)
        if beta <= 0.1:
            countoutL+=1

    elif name in ecc:
        line2 = ax4.scatter(beta, sopt, c='black', s=10)
        if beta <= 0.1:
            countoutL+=1
    elif D>28e-3:
        line3 = ax4.scatter(beta, sopt, c='darkorange', s=10)
        if beta <= 0.1:
            countoutL+=1
    else:
        ax4.scatter(beta, sopt, c='blue', s=3)

plt.legend([line1, line2, line3], 
           [r'$\mathrm{q \geq 1.5}$',
            r'highly eccentric', 
            r'$\mathrm{dur_{postm} \geq 30ms}$'], loc='upper right',
          fontsize='x-small') 

plt.xlabel(r'$\beta=\mathrm{\frac{d_{best}}{d_{postm}}}$', fontsize=20) 
plt.ylabel(r'$\mathrm{S_{opt}}$'
           , fontsize=20)

#plt.title(f'{count20} waveforms, out of which {countoutL/count20 * 100:.0f} percent are either...')
plt.axvline(vline, linestyle='dashed', color='red');
plt.xlim(0, 1.3)

plt.tight_layout()
#plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/alpha_sum1.pdf')


In [ ]:
fig4 = plt.figure(dpi=1200)
#ax4 = fig4.add_subplot()#2,1,1)
ax5 = fig4.add_subplot()#2,1,2)

vline = 0.2
count30 = 0
countoutL = 0
for name in JOINT:
    sopt = JOINT[name]['SNR_opt'][0]
    srec = JOINT[name]['SNR_rec'][0]
    
    D = JOINT[name]['wf_d'][0]
    d_b = JOINT[name]['best_d'][0]
    
    
    Fw = JOINT[name]['wf_f_wei'][0]
    f_b = JOINT[name]['best_f'][0]
    
    gamma = f_b/Fw      
    beta = d_b/D
    
    q = JOINT[name]['mdta']['M_r'][0]


    if abs(1-gamma) > vline:
        count30+=1
        
    if D<=5e-3:
        line1 = ax5.scatter(gamma, sopt, c='green', 
                    marker='x', s=10) 
        if abs(1-gamma) > vline:
            countoutL+=1
    elif D>30e-3:
        line4 = ax5.scatter(gamma, sopt, c='darkorange', 
                    marker='x', s=10) 
        if abs(1-gamma) > vline:
            countoutL+=1
    
    elif q>=1.5: 
        line2 = ax5.scatter(gamma, sopt, c='r', marker='v', s=10)
        if abs(1-gamma) > vline:
            countoutL+=1

    elif name in ecc:

        line3 = ax5.scatter(gamma, sopt, c='black', s=10)
        if abs(1-gamma) > vline:
            countoutL+=1
    else:
        ax5.scatter(gamma, sopt, c='blue', s=10)

plt.legend([line1, line2, line3, line4], 
           [r'$\mathrm{d_{postm}\leq 5ms}$',
            r'$\mathrm{q \geq 1.5}$',
            r'highly eccentric',
           r'$\mathrm{d_{postm}\ge 30ms}$'], loc='upper right',
          fontsize='x-small') 

plt.xlabel(r'$\mathrm{\gamma=\frac{f_{best}}{f_{wei}}}$', fontsize=20) 
plt.ylabel(r'$\mathrm{S_{opt}}$', fontsize=20)

plt.title(f'{count30} waveforms, out of which {countoutL/count30 * 100:.0f} percent are either...')
plt.axvline(1-vline, linestyle='dashed', color='red');
plt.axvline(1+vline, linestyle='dashed', color='red');

plt.tight_layout()
#plt.savefig('/home/gthth/REPOS1/sebas-thesis/images/Data_analysis/results/alpha_sum2.pdf')
